The same as `multitask-pretraining`, but initialized from a raw BART for fair evaluation in both transfer directions.

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
from datasets import load_dataset

# Collect the training tasks

* Paraphrasing: https://huggingface.co/datasets/GEM/opusparcus
* Translation: https://huggingface.co/datasets/open_subtitles + news_commentary? + tatoeba?
* Detox: ordinary data

In [3]:
opus_para_en = load_dataset("GEM/opusparcus", "en.80")
opus_para_en

Reusing dataset opusparcus (/home/dale/.cache/huggingface/datasets/GEM___opusparcus/en.80/1.0.0/79d36ae4eced4f3c2c5a2ab9f94a584de7adca957186408d33798d0d87b018f2)


  0%|          | 0/5 [00:00<?, ?it/s]

DatasetDict({
    test: Dataset({
        features: ['lang', 'input', 'target', 'annot_score', 'gem_id', 'references'],
        num_rows: 982
    })
    validation: Dataset({
        features: ['lang', 'input', 'target', 'annot_score', 'gem_id', 'references'],
        num_rows: 1015
    })
    test.full: Dataset({
        features: ['lang', 'input', 'target', 'annot_score', 'gem_id', 'references'],
        num_rows: 1445
    })
    validation.full: Dataset({
        features: ['lang', 'input', 'target', 'annot_score', 'gem_id', 'references'],
        num_rows: 1455
    })
    train: Dataset({
        features: ['lang', 'input', 'target', 'annot_score', 'gem_id', 'references'],
        num_rows: 5200000
    })
})

In [4]:
opus_para_ru = load_dataset("GEM/opusparcus", "ru.80")
opus_para_ru

Reusing dataset opusparcus (/home/dale/.cache/huggingface/datasets/GEM___opusparcus/ru.80/1.0.0/79d36ae4eced4f3c2c5a2ab9f94a584de7adca957186408d33798d0d87b018f2)


  0%|          | 0/5 [00:00<?, ?it/s]

DatasetDict({
    test: Dataset({
        features: ['lang', 'input', 'target', 'annot_score', 'gem_id', 'references'],
        num_rows: 1068
    })
    validation: Dataset({
        features: ['lang', 'input', 'target', 'annot_score', 'gem_id', 'references'],
        num_rows: 1020
    })
    test.full: Dataset({
        features: ['lang', 'input', 'target', 'annot_score', 'gem_id', 'references'],
        num_rows: 1855
    })
    validation.full: Dataset({
        features: ['lang', 'input', 'target', 'annot_score', 'gem_id', 'references'],
        num_rows: 1854
    })
    train: Dataset({
        features: ['lang', 'input', 'target', 'annot_score', 'gem_id', 'references'],
        num_rows: 2300000
    })
})

In [5]:
import random

In [6]:
random.choice(opus_para_ru['train'])

{'lang': 'ru',
 'input': 'На-ка.',
 'target': 'Получи.',
 'annot_score': 0.0,
 'gem_id': 'gem-opusparcus-train-79939030',
 'references': ['Получи.']}

In [7]:
opensub = load_dataset("open_subtitles", lang1="en", lang2='ru')
opensub

Using custom data configuration en-ru-lang1=en,lang2=ru
Reusing dataset open_subtitles (/home/dale/.cache/huggingface/datasets/open_subtitles/en-ru-lang1=en,lang2=ru/0.0.0/c1ec973ca4b6e588740d8f167cc0e24ea3f626e70bc7ffe467e944730500e198)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'meta', 'translation'],
        num_rows: 25910105
    })
})

In [8]:
random.choice(opensub['train'])

{'id': '2752315',
 'meta': {'year': 1993,
  'imdbId': 701066,
  'subtitleId': {'en': 5591101, 'ru': 7006084},
  'sentenceIds': {'en': [118], 'ru': [105]}},
 'translation': {'en': 'Observe.', 'ru': 'Велосипедный замок.'}}

In [9]:
news_commentary = load_dataset("news_commentary", lang1="en", lang2='ru')
news_commentary

Using custom data configuration en-ru-lang1=en,lang2=ru
Reusing dataset news_commentary (/home/dale/.cache/huggingface/datasets/news_commentary/en-ru-lang1=en,lang2=ru/0.0.0/cfab724ce975dc2da51cdae45302389860badc88b74db8570d561ced6004f8b4)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 190104
    })
})

In [10]:
random.choice(news_commentary['train'])

{'id': '87720',
 'translation': {'en': 'How a concept is framed, the context and associations with which it is presented, affect human judgments enormously.',
  'ru': 'Обрамление, контекст и ассоциации, с которыми связано то или иное понятие, очень сильно влияют на суждения людей.'}}

In [11]:
tatoeba = load_dataset("tatoeba", lang1="en", lang2='ru')
tatoeba

Using custom data configuration en-ru-lang1=en,lang2=ru
Reusing dataset tatoeba (/home/dale/.cache/huggingface/datasets/tatoeba/en-ru-lang1=en,lang2=ru/0.0.0/b3ea9c6bb2af47699c5fc0a155643f5a0da287c7095ea14824ee0a8afd74daf6)


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 523656
    })
})

In [12]:
random.choice(tatoeba['train'])

{'id': '391340',
 'translation': {'en': "I still don't understand what you're talking about.",
  'ru': 'Я так и не понял, о чём ты.'}}

In [13]:
def get_paraphrase_task(batch_size=1):
    task = 'paraphrase: '
    if random.random() < 0.5:
        src = opus_para_en
        src_id, tgt_id = 'en_XX', 'en_XX'
    else:
        src = opus_para_ru
        src_id, tgt_id = 'ru_RU', 'ru_RU'
    x = []
    y = []
    for _ in range(batch_size):
        item = random.choice(src['train'])
        xx, yy = item['input'], item['target']
        if random.random() < 0.5:
            xx, yy = yy, xx
        x.append(task + xx)
        y.append(yy)
    return x, y, src_id, tgt_id
        
get_paraphrase_task(1)

(['paraphrase: Влезай.'], ['Входи.'], 'ru_RU', 'ru_RU')

In [14]:
def get_translate_task(batch_size=1):
    task = 'translate: '
    src = random.choice([tatoeba, opensub, news_commentary])
    src_id, tgt_id = 'ru_RU', 'en_XX'
    if random.random() < 0.5:
        src_id, tgt_id = tgt_id, src_id
    x = []
    y = []
    for _ in range(batch_size):
        item = random.choice(src['train'])
        x.append(task + item['translation'][src_id[:2]])
        y.append(item['translation'][tgt_id[:2]])
    return x, y, src_id, tgt_id

get_translate_task(2)

(['translate: I think Tom really likes Mary.',
  'translate: Tom caught three fish yesterday.'],
 ['Я думаю, Мэри действительно нравится Тому.', 'Вчера Том поймал три рыбы.'],
 'en_XX',
 'ru_RU')

In [15]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [38]:
def paraphrase(
    text, model, tokenizer, 
    n=None, 
    max_length="auto", 
    beams=5,
    src_lang='en_XX',
    tgt_lang='en_XX',
    **kwargs
):
    tokenizer.src_lang = src_lang
    tokenizer.tgt_lang = tgt_lang
    texts = [text] if isinstance(text, str) else text
    tokenizer.tgt_lang
    inputs = tokenizer(texts, return_tensors="pt", padding=True)["input_ids"].to(
        model.device
    )

    if max_length == "auto":
        max_length = inputs.shape[1] + 10

    result = model.generate(
        inputs,
        num_return_sequences=n or 1,
        do_sample=False,
        temperature=1.0,
        repetition_penalty=10.0,
        max_length=max_length,
        min_length=int(0.5 * max_length),
        num_beams=beams,
        forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_lang),
    )
    texts = [tokenizer.decode(r, skip_special_tokens=True) for r in result]

    if not n and isinstance(text, str):
        return texts[0]
    return texts[0]

In [16]:
model_name = 'facebook/mbart-large-50'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [17]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).cuda();

In [18]:
def get_batch(batch_size=4):
    x = []
    y = []
    bs = batch_size // 2
    for generator in [get_paraphrase_task, get_translate_task]:
        for _ in range(bs):
            xx, yy, src_id, tgt_id = generator(1)
            tokenizer.src_lang, tokenizer.tgt_lang = src_id, tgt_id
            x.append(tokenizer(xx))
            with tokenizer.as_target_tokenizer():
                y.append(tokenizer(yy))
    return x, y

In [19]:
x, y = get_batch(6)
x, y

([{'input_ids': [[250021, 121, 179665, 184, 12, 1089, 695, 5, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1]]},
  {'input_ids': [[250021, 121, 179665, 184, 12, 536, 49, 3920, 29732, 103, 162991, 32, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]},
  {'input_ids': [[250021, 121, 179665, 184, 12, 16434, 15303, 49, 77, 197709, 4544, 5, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]},
  {'input_ids': [[250021, 3900, 19309, 12, 7218, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1]]},
  {'input_ids': [[250004, 3900, 19309, 12, 345, 6953, 70, 21, 3055, 6626, 52895, 47, 6, 178451, 44, 15663, 10821, 130815, 7, 58, 98, 101334, 7, 1379, 2962, 7, 171827, 3098, 14202, 25, 144189, 2481, 50509, 214, 17265, 21, 24658, 214, 4, 4127, 23131, 7154, 4, 3900, 16082, 27771, 4, 23991, 80244, 4, 136, 111789, 165439, 102880, 46518, 5844, 47, 25313, 75678, 5, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [20]:
for xx, yy in zip(x, y):
    print(tokenizer.decode(xx['input_ids'][0]))
    print(tokenizer.decode(yy['input_ids'][0]))

ru_RU paraphrase: Ого.</s>
ru_RU Ооо.</s>
ru_RU paraphrase: А в этом дереве золото?</s>
ru_RU Скажи что-нибудь по-английски.</s>
ru_RU paraphrase: Ты здесь в небезопасности.</s>
ru_RU Здесь не безопасно.</s>
ru_RU translate: 49</s>
en_XX 49%</s>
en_XX translate: Using the latter two types to impose "odious debts" on Africans undermines western lenders' credibility concerning money laundering, good governance, transparency, fiscal discipline, and macroeconomic policies conducive to economic growth.</s>
ru_RU Использование последних двух типов с целью обложить африканцев “ненавистными долгами” подрывает доверие к западным кредиторам относительно отмывания денег, хорошего управления, прозрачности, финансового порядка и макроэкономической политики, способствующих экономическому росту.</s>
ru_RU translate: Выигрышный бросок Путина</s>
en_XX Putin’s Winning Streak</s>


In [21]:
tokenizer.pad([{k: v[0] for k, v in item.items()} for item in x])

{'input_ids': [[250021, 121, 179665, 184, 12, 1089, 695, 5, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [250021, 121, 179665, 184, 12, 536, 49, 3920, 29732, 103, 162991, 32, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [250021, 121, 179665, 184, 12, 16434, 15303, 49, 77, 197709, 4544, 5, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [250021, 3900, 19309, 12, 7218, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [250004, 3900, 19309, 12, 345, 6953, 70, 21, 3055, 6626, 52895, 47, 6, 178451, 44, 15663, 10821, 130815, 7, 58, 98, 101334, 7, 1379, 2962, 7, 1

In [22]:
import transformers
transformers.logging.set_verbosity_error()

In [23]:
from transformers.optimization import Adafactor
optimizer = Adafactor(model.parameters(), scale_parameter=False, relative_step=False, lr=1e-5, clip_threshold=1.0)

In [24]:
from torch.optim.lr_scheduler import LambdaLR

num_warmup_steps = 1000

def lr_lambda(current_step: int):
    if current_step < num_warmup_steps:
        return float(current_step) / float(max(1.0, num_warmup_steps))
    return 1.0

scheduler = LambdaLR(optimizer, lr_lambda)

In [25]:
model.train()
model.gradient_checkpointing_enable()

In [26]:
from tqdm.auto import tqdm, trange

In [27]:
import torch
import gc

def cleanup():
    gc.collect()
    torch.cuda.empty_cache()

cleanup()

In [28]:
gradient_steps = 4
batch_size = 6
window = 1000
report_steps = 1000
cleanup_steps = 100
save_path = '/home/dale/models/detox-parallel/bart-multitask-pretrain-clean'

In [29]:
ewm_loss = 4

In [30]:
cleanup()

With ~6 iterations per second, 1M iteration takes 1000000/6/60/60=45 hours.

In [ ]:
model.train()
tq = trange(0 , 1_000_000)
cleanup()

for i in tq:
    x, y = get_batch(batch_size)
    xpad = tokenizer.pad([{k: v[0] for k, v in item.items()} for item in x])
    ypad = tokenizer.pad([{k: v[0] for k, v in item.items()} for item in y])
    
    try:
        labels = torch.tensor(ypad['input_ids'], device=model.device)
        labels[labels==tokenizer.pad_token_id] = -100
        loss = model(
            input_ids=torch.tensor(xpad['input_ids'], device=model.device),
            attention_mask=torch.tensor(xpad['attention_mask'], device=model.device),
            labels=labels,
        ).loss
        loss.backward()
        if i % gradient_steps == 0:
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)
            scheduler.step()
    except RuntimeError as e:
        print('error', i, e)
        loss = None
        optimizer.zero_grad(set_to_none=True)
        cleanup()
        continue

    w = 1 / max(1, min(i, window))
    ewm_loss = ewm_loss * (1-w) + loss.item() * w
    tq.set_description(f'{ewm_loss:3.4f}')

    if i > 0 and i % report_steps == 0:
        print('step', i, 'loss', ewm_loss, )
        if i > 0:
            model.save_pretrained(save_path)
            tokenizer.save_pretrained(save_path)
    if i % cleanup_steps == 0:
        cleanup()

    

  0%|          | 0/1000000 [00:00<?, ?it/s]

step 1000 loss 7.685543497323983
step 2000 loss 6.70197135323358
error 2682 CUDA out of memory. Tried to allocate 1.05 GiB (GPU 0; 10.76 GiB total capacity; 8.48 GiB already allocated; 983.69 MiB free; 8.67 GiB reserved in total by PyTorch)
step 3000 loss 4.517895718230879
step 4000 loss 3.0584388129091327
step 5000 loss 2.369700513029202
step 6000 loss 2.0188946715580154
error 6212 CUDA out of memory. Tried to allocate 978.00 MiB (GPU 0; 10.76 GiB total capacity; 7.30 GiB already allocated; 759.69 MiB free; 8.89 GiB reserved in total by PyTorch)
error 6243 CUDA out of memory. Tried to allocate 1.05 GiB (GPU 0; 10.76 GiB total capacity; 8.48 GiB already allocated; 555.69 MiB free; 9.09 GiB reserved in total by PyTorch)
error 6971 CUDA out of memory. Tried to allocate 1002.00 MiB (GPU 0; 10.76 GiB total capacity; 8.22 GiB already allocated; 715.69 MiB free; 8.94 GiB reserved in total by PyTorch)
step 7000 loss 1.8909473272931656
error 7248 CUDA out of memory. Tried to allocate 870.00 Mi

In [67]:
print(i, ewm_loss)
# step 141000 loss 0.9336458999525007
# 598943 0.7913156676625156
# 999999 0.7528545096685079

49999 0.7369652006657895


In [ ]:
model.eval();

In [ ]:
tokenizer.src_lang = 'ru_RU'

In [ ]:
tokenizer.src_lang = 'en_XX'

In [ ]:
tokenizer('привет')

In [ ]:
tokenizer.convert_tokens_to_ids('en_XX')

In [ ]:
print(paraphrase('paraphrase: I like to play with my nice dog.', model, tokenizer))

In [ ]:
print(paraphrase('translate: I like to play with my nice dog.', model, tokenizer, tgt_lang='ru_RU'))

In [ ]:
print(paraphrase('translate: I hate to play with my fucking dog.', model, tokenizer, tgt_lang='ru_RU'))

In [ ]:
print(paraphrase('translate: Ненавижу играть со своей долбаной собакой.', model, tokenizer, src_lang='ru_RU'))

In [ ]:
print(paraphrase('paraphrase: I hate to play with my fucking dog.', model, tokenizer, tgt_lang='en_XX'))

# Transfer from English to Russian

In [ ]:
import pandas as pd

In [ ]:
detox_en2ru = pd.read_csv('detox_en2ru_yandex.tsv', sep='\t')

In [ ]:
detox_en2ru_filtered = detox_en2ru[
    (detox_en2ru.edit_distance_ru >= detox_en2ru.edit_distance_en.quantile(0.01)) 
    & (detox_en2ru.edit_distance_ru <= detox_en2ru.edit_distance_en.quantile(0.99)) 
    & (detox_en2ru.edit_sim_ru >= detox_en2ru.edit_sim_en.quantile(0.01)) 
    & (detox_en2ru.edit_sim_ru <= detox_en2ru.edit_sim_en.quantile(0.99))
]

print(detox_en2ru.shape)
print(detox_en2ru_filtered.shape)

In [ ]:
detox_en2ru_filtered.sample(3)

In [ ]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(detox_en2ru_filtered, random_state=1, test_size=500)

In [ ]:
def get_detox_task_en(batch_size=1):
    task = 'detox: '
    sample = train.sample(batch_size)
    src_id, tgt_id = 'en_XX', 'en_XX'
    x = []
    y = []
    for row in sample.itertuples():
        x.append(task + row.toxic_comment)
        y.append(row.neutral_comment)
    return x, y, src_id, tgt_id

get_detox_task_en(2)

In [ ]:
def get_detox_task_ru(batch_size=1):
    task = 'detox: '
    sample = train.sample(batch_size)
    src_id, tgt_id = 'ru_RU', 'ru_RU'
    x = []
    y = []
    for row in sample.itertuples():
        x.append(task + row.toxic_ru)
        y.append(row.neutral_ru)
    return x, y, src_id, tgt_id

get_detox_task_ru(2)

In [ ]:
def get_batch(batch_size=4):
    x = []
    y = []
    bs = batch_size // 4
    for generator in [get_detox_task_en, get_detox_task_ru, get_paraphrase_task, get_translate_task]:
        for _ in range(bs):
            xx, yy, src_id, tgt_id = generator(1)
            tokenizer.src_lang, tokenizer.tgt_lang = src_id, tgt_id
            x.append(tokenizer(xx))
            with tokenizer.as_target_tokenizer():
                y.append(tokenizer(yy))
    return x, y

In [ ]:
!ls /home/dale/models/detox-parallel

In [ ]:
batch_size = 8
save_path = '/home/dale/models/detox-parallel/bart-multitask-v4-ru'

In [ ]:
model.train()
tq = trange(0, 50_000)
cleanup()

for i in tq:
    x, y = get_batch(batch_size)
    xpad = tokenizer.pad([{k: v[0] for k, v in item.items()} for item in x])
    ypad = tokenizer.pad([{k: v[0] for k, v in item.items()} for item in y])
    
    try:
        labels = torch.tensor(ypad['input_ids'], device=model.device)
        labels[labels==tokenizer.pad_token_id] = -100
        loss = model(
            input_ids=torch.tensor(xpad['input_ids'], device=model.device),
            attention_mask=torch.tensor(xpad['attention_mask'], device=model.device),
            labels=labels,
        ).loss
        loss.backward()
        if i % gradient_steps == 0:
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)
            scheduler.step()
    except RuntimeError as e:
        print('error', i, e)
        loss = None
        optimizer.zero_grad(set_to_none=True)
        cleanup()
        continue

    w = 1 / max(1, min(i, window))
    ewm_loss = ewm_loss * (1-w) + loss.item() * w
    tq.set_description(f'{ewm_loss:3.4f}')

    if i > 0 and i % report_steps == 0:
        print('step', i, 'loss', ewm_loss, )
        if i > 0:
            model.save_pretrained(save_path)
            tokenizer.save_pretrained(save_path)
    if i % cleanup_steps == 0:
        cleanup()

In [ ]:
model.eval();

In [ ]:
print(paraphrase('paraphrase: I like to play with my nice dog.', model, tokenizer))

In [ ]:
print(paraphrase('translate: I like to play with my nice dog.', model, tokenizer, tgt_lang='ru_RU'))

In [ ]:
print(paraphrase('translate: I hate to play with my fucking dog.', model, tokenizer, tgt_lang='ru_RU'))

In [ ]:
print(paraphrase('translate: Ненавижу играть со своей долбаной собакой.', model, tokenizer, src_lang='ru_RU'))

In [ ]:
print(paraphrase('paraphrase: I hate to play with my fucking dog.', model, tokenizer, tgt_lang='en_XX'))

In [ ]:
print(paraphrase('detox: I hate to play with my fucking dog.', model, tokenizer, tgt_lang='en_XX'))

In [ ]:
print(paraphrase('detox: Ненавижу играть со своей долбаной собакой.', model, tokenizer, src_lang='ru_RU', tgt_lang='ru_RU'))

In [ ]:
test_data = pd.read_csv('../data/russian_data/test.tsv', sep='\t')
test_inputs = test_data["toxic_comment"].values.tolist()

In [ ]:
test_outputs = [paraphrase('detox: ' + text, model, tokenizer, src_lang='ru_RU', tgt_lang='ru_RU') for text in tqdm(test_inputs)]

In [ ]:
path = '../results/multitask-1M-translate-yandex-v4/' 
if not os.path.exists(path):
    os.mkdir(path)

In [ ]:
with open(path + 'results_ru.txt', 'w') as f:
    for line in test_outputs:
        f.write(line + '\n')

Full results (1m steps)  + 50K with Russian translated data
```
cd /home/dale/projects/multilingual_detox
python evaluate_ru.py \
    --result_filename scores \
    --input_dir results/multitask-1M-translate-yandex-v4 \
    --output_dir results

Style accuracy:       0.6400219798088074
Meaning preservation: 0.8625849485397339
Joint fluency:        -0.09933383762836456
Joint score:          -0.052584096789360046
Scores after calibration:
Style accuracy:       0.6760197877883911
Meaning preservation: 0.7941577434539795
Joint fluency:        0.8857660889625549
Joint score:          0.47143980860710144

```

In [71]:
with open('../data/english_data/test_toxic_parallel.txt', 'r') as f:
    test_inputs_en = [line.strip() for line in f.readlines()]

In [72]:
with open('../data/english_data/test_toxic_parallel.txt', 'r') as f:
    test_inputs_en = [line.strip() for line in f.readlines()]

In [74]:
test_outputs_en = [paraphrase('detox: ' + text, model, tokenizer, src_lang='en_XX', tgt_lang='en_XX') for text in tqdm(test_inputs_en)]

  0%|          | 0/671 [00:00<?, ?it/s]

In [77]:
with open(path + 'results_en.txt', 'w') as f:
    for line in test_outputs_en:
        f.write(line + '\n')

```
cd /home/dale/projects/paradetox2/evaluation_detox
python metric.py --inputs /home/dale/projects/multilingual_detox/data/english_data/test_toxic_parallel.txt \
    --preds /home/dale/projects/multilingual_detox/results/multitask-1M-translate-yandex-v4/results_en.txt \
    --cola_classifier_path /home/dale/models/cola_classifier_fairseq \
    --wieting_model_path /home/dale/models/wieting_similarity/sim.pt \
    --wieting_tokenizer_path /home/dale/models/wieting_similarity/sim.sp.30k.model \
    --batch_size 32
cat results.md
```

| Model | ACC | EMB_SIM | SIM | CharPPL | TokenPPL | FL | GM | J | BLEU |
| ----- | --- | ------- | --- | ------- | -------- | -- | -- | - | ---- |
results_en.txt|0.8852|0.9024|0.8583|6.5192|145.7229|0.8689|11.4473|0.6523|0.7337|

# Transfer from Russian to English

In [29]:
import pandas as pd
from tqdm.auto import tqdm, trange

import torch
import gc

def cleanup():
    gc.collect()
    torch.cuda.empty_cache()

cleanup()

In [16]:
model_name = '/home/dale/models/detox-parallel/bart-multitask-pretrain-clean'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [17]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).cuda();

In [20]:
detox_ru2en = pd.read_csv('detox_ru2en_yandex.tsv', sep='\t')

In [21]:
detox_ru2en_filtered = detox_ru2en[
    ((detox_ru2en.edit_distance_en >= detox_ru2en.edit_distance_ru.quantile(0.01)) 
    & (detox_ru2en.edit_distance_en <= detox_ru2en.edit_distance_ru.quantile(0.99)) 
    & (detox_ru2en.edit_sim_en >= detox_ru2en.edit_sim_ru.quantile(0.01)) 
    & (detox_ru2en.edit_sim_en <= detox_ru2en.edit_sim_ru.quantile(0.99))
    )
]

print(detox_ru2en.shape)
print(detox_ru2en_filtered.shape)

(7058, 16)
(6739, 16)


In [22]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(detox_ru2en_filtered, random_state=1, test_size=500)
print(train.shape)
print(val.shape)

(6239, 16)
(500, 16)


In [23]:
train.sample(3)

,idx,toxic_comment,neutral_comment,toxicity_score,dataset,toxic,confidence_toxic,is_match,confidence_is_match,split,toxic_en,neutral_en,edit_distance_ru,edit_distance_en,edit_sim_ru,edit_sim_en
3830,6075,убить иее мала пусть бог иее рассудит,Пусть бог её рассудит,0.999279,ru,False,0.9999,True,0.9839,train,kill iee small let god judge iee,May God judge her,18,20,0.513514,0.375000
4426,12772,"в поезд не сядет тот, кто помнит время ссср то...","В поезд не сядет тот, кто помнит время при гор...",0.999282,ru,False,0.9620,True,0.9882,train,the one who remembers the time of the USSR onl...,No one who remembers the time under Gorbachev ...,32,40,0.614458,0.619048
5663,10432,"не смотрите, меня хватило на 30 минут этой мут...","Не смотрите, у меня хватило терпения на 30 мин...",0.991059,ru,False,0.9689,True,0.9915,dev,"don't look, I had enough for 30 minutes of thi...","Don't look, I had enough patience for 30 minut...",40,24,0.452055,0.666667


In [24]:
def get_detox_task_en(batch_size=1):
    task = 'detox: '
    sample = train.sample(batch_size)
    src_id, tgt_id = 'en_XX', 'en_XX'
    x = []
    y = []
    for row in sample.itertuples():
        x.append(task + row.toxic_en)
        y.append(row.neutral_en)
    return x, y, src_id, tgt_id

get_detox_task_en(2)

(['detox: God, what kind of fuckers are there in the world. ..',
  'detox: to be honest, the shit is rare'],
 ['My God, who is not in this world', "To be honest, it's rare nonsense"],
 'en_XX',
 'en_XX')

In [25]:
def get_detox_task_ru(batch_size=1):
    task = 'detox: '
    sample = train.sample(batch_size)
    src_id, tgt_id = 'ru_RU', 'ru_RU'
    x = []
    y = []
    for row in sample.itertuples():
        x.append(task + row.toxic_comment)
        y.append(row.neutral_comment)
    return x, y, src_id, tgt_id

get_detox_task_ru(2)

(['detox: никому не нужны чужие выблядки.',
  'detox: суки опять какую то хуйню спустили в реку уебки'],
 ['Никому не нужны чужие дети', 'Опять что-то в реку спустили'],
 'ru_RU',
 'ru_RU')

In [26]:
def get_batch(batch_size=4):
    x = []
    y = []
    bs = batch_size // 4
    for generator in [get_detox_task_en, get_detox_task_ru, get_paraphrase_task, get_translate_task]:
        for _ in range(bs):
            xx, yy, src_id, tgt_id = generator(1)
            tokenizer.src_lang, tokenizer.tgt_lang = src_id, tgt_id
            x.append(tokenizer(xx))
            with tokenizer.as_target_tokenizer():
                y.append(tokenizer(yy))
    return x, y

In [30]:
gradient_steps = 4
window = 1000
report_steps = 1000
cleanup_steps = 100

In [27]:
batch_size = 8
save_path = '/home/dale/models/detox-parallel/bart-multitask-v4-en'
save_path

'/home/dale/models/detox-parallel/bart-multitask-v4-en'

In [32]:
from transformers.optimization import Adafactor
optimizer = Adafactor(model.parameters(), scale_parameter=False, relative_step=False, lr=1e-5, clip_threshold=1.0)

In [34]:
ewm_loss = 4

In [35]:
model.train()
tq = trange(0, 50_000)
cleanup()

for i in tq:
    x, y = get_batch(batch_size)
    xpad = tokenizer.pad([{k: v[0] for k, v in item.items()} for item in x])
    ypad = tokenizer.pad([{k: v[0] for k, v in item.items()} for item in y])
    
    try:
        labels = torch.tensor(ypad['input_ids'], device=model.device)
        labels[labels==tokenizer.pad_token_id] = -100
        loss = model(
            input_ids=torch.tensor(xpad['input_ids'], device=model.device),
            attention_mask=torch.tensor(xpad['attention_mask'], device=model.device),
            labels=labels,
        ).loss
        loss.backward()
        if i % gradient_steps == 0:
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)
            #scheduler.step()
    except RuntimeError as e:
        print('error', i, e)
        loss = None
        optimizer.zero_grad(set_to_none=True)
        cleanup()
        continue

    w = 1 / max(1, min(i, window))
    ewm_loss = ewm_loss * (1-w) + loss.item() * w
    tq.set_description(f'{ewm_loss:3.4f}')

    if i > 0 and i % report_steps == 0:
        print('step', i, 'loss', ewm_loss, )
        if i > 0:
            model.save_pretrained(save_path)
            tokenizer.save_pretrained(save_path)
    if i % cleanup_steps == 0:
        cleanup()

  0%|          | 0/50000 [00:00<?, ?it/s]

error 72 CUDA out of memory. Tried to allocate 990.00 MiB (GPU 0; 10.76 GiB total capacity; 8.05 GiB already allocated; 739.69 MiB free; 8.91 GiB reserved in total by PyTorch)
step 1000 loss 1.2333002176196652
step 2000 loss 1.172667905884665
step 3000 loss 1.1308728053043728
step 4000 loss 1.0866697279344393
step 5000 loss 1.0693783238909067
step 6000 loss 1.0342307965602748
step 7000 loss 1.0176406868389598
step 8000 loss 1.00346553988644
step 9000 loss 0.9658832415699443
step 10000 loss 0.9669977334382361
step 11000 loss 0.9555475747068011
step 12000 loss 0.9293789452994402
step 13000 loss 0.9130843327986824
step 14000 loss 0.8970300269012222
step 15000 loss 0.8924606968906944
step 16000 loss 0.8660455307541913
step 17000 loss 0.8734970424154661
step 18000 loss 0.8545141288301037
step 19000 loss 0.8468137140176221
step 20000 loss 0.8327704592709183
step 21000 loss 0.8169362085610442
step 22000 loss 0.8032452978234469
step 23000 loss 0.7969312192213301
step 24000 loss 0.7958858616933

In [36]:
model.eval();

In [39]:
print(paraphrase('paraphrase: I like to play with my nice dog.', model, tokenizer))

I like to play with my dog, you know?


In [40]:
print(paraphrase('translate: I like to play with my nice dog.', model, tokenizer, tgt_lang='ru_RU'))

Мне нравится играть со своей милой собакой.


In [41]:
print(paraphrase('translate: I hate to play with my fucking dog.', model, tokenizer, tgt_lang='ru_RU'))

Ненавижу играть со своей собакой.


In [42]:
print(paraphrase('translate: Ненавижу играть со своей долбаной собакой.', model, tokenizer, src_lang='ru_RU'))

I hate to play with my dog, you bastard.


In [43]:
print(paraphrase('paraphrase: I hate to play with my fucking dog.', model, tokenizer, tgt_lang='en_XX'))

I don't want to play with my dog.


In [44]:
print(paraphrase('detox: I hate to play with my fucking dog.', model, tokenizer, tgt_lang='en_XX'))

I don't like to play with my dog


In [45]:
print(paraphrase('detox: Ненавижу играть со своей долбаной собакой.', model, tokenizer, src_lang='ru_RU', tgt_lang='ru_RU'))

Я ненавижу играть со своей собакой.


In [46]:
test_data = pd.read_csv('../data/russian_data/test.tsv', sep='\t')
test_inputs = test_data["toxic_comment"].values.tolist()

In [47]:
test_outputs = [paraphrase('detox: ' + text, model, tokenizer, src_lang='ru_RU', tgt_lang='ru_RU') for text in tqdm(test_inputs)]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [48]:
path = '../results/multitask-1M-translate-yandex-v4-ru2en/' 
if not os.path.exists(path):
    os.mkdir(path)

In [49]:
with open(path + 'results_ru.txt', 'w') as f:
    for line in test_outputs:
        f.write(line + '\n')

Full results (1m steps)  + 50K with Russian translated data
```
cd /home/dale/projects/multilingual_detox
python evaluate_ru.py \
    --result_filename scores \
    --input_dir results/multitask-1M-translate-yandex-v4-ru2en \
    --output_dir results

Style accuracy:       0.7263924479484558
Meaning preservation: 0.8321331739425659
Joint fluency:        -0.1356080323457718
Joint score:          -0.07764336466789246
Scores after calibration:
Style accuracy:       0.7537532448768616
Meaning preservation: 0.7489004135131836
Joint fluency:        0.8440507650375366
Joint score:          0.4865816831588745

```

In [50]:
with open('../data/english_data/test_toxic_parallel.txt', 'r') as f:
    test_inputs_en = [line.strip() for line in f.readlines()]

In [51]:
test_outputs_en = [paraphrase('detox: ' + text, model, tokenizer, src_lang='en_XX', tgt_lang='en_XX') for text in tqdm(test_inputs_en)]

  0%|          | 0/671 [00:00<?, ?it/s]

In [52]:
with open(path + 'results_en.txt', 'w') as f:
    for line in test_outputs_en:
        f.write(line + '\n')

```
cd /home/dale/projects/paradetox2/evaluation_detox
python metric.py --inputs /home/dale/projects/multilingual_detox/data/english_data/test_toxic_parallel.txt \
    --preds /home/dale/projects/multilingual_detox/results/multitask-1M-translate-yandex-v4-ru2en/results_en.txt \
    --cola_classifier_path /home/dale/models/cola_classifier_fairseq \
    --wieting_model_path /home/dale/models/wieting_similarity/sim.pt \
    --wieting_tokenizer_path /home/dale/models/wieting_similarity/sim.sp.30k.model \
    --batch_size 32
cat results.md
```

| Model | ACC | EMB_SIM | SIM | CharPPL | TokenPPL | FL | GM | J | BLEU |
| ----- | --- | ------- | --- | ------- | -------- | -- | -- | - | ---- |
results_en.txt|0.7273|0.8489|0.8193|6.0657|104.0004|0.8838|0.0000|0.5032|0.6794|